# Amazon SageMaker와 MLflow 설정 방법

## Updates and Imports

In [ ]:
import json
import sagemaker
import boto3
import time
from botocore.exceptions import ClientError

Session variables

In [ ]:
sess = sagemaker.Session()
bucket_name = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

iam_client = boto3.client("iam")
sts_client = boto3.client("sts")
sm_client = boto3.client("sagemaker")
account_id = sts_client.get_caller_identity()["Account"]
tracking_server_name = "my-setup-test3"
mlflow_role_name = "mlflow-test3"

## MLflow Permissions

# MLflow 추적 서버를 위한 IAM 역할

이 다음 셀을 실행하려면 이 노트북을 실행하는 데 사용되는 IAM 역할에 IAM 역할을 생성할 수 있는 권한이 있는지 확인하세요.
노트북 실행 역할의 정책에서 `iam:CreateRole`, `iam:CreatePolicy`, `iam:ListPolicies` 및 `iam:AttachRolePolicy` 작업이 허용되어야 합니다.

SageMaker Studio에서 이 노트북을 실행하는 경우, 다음 단계를 통해 노트북 실행 역할을 업데이트할 수 있습니다:

1. AWS 콘솔로 이동하여 사용 중인 도메인을 선택합니다
2. 도메인 아래에서 사용 중인 사용자 프로필을 선택합니다. 실행 역할이 거기에 나열되어 있을 것입니다.
3. IAM 콘솔로 이동하여 "역할" 아래에서 실행 역할을 검색하고, `iam:CreateRole`, `iam:CreatePolicy`, `iam:ListPolicies` 및 `iam:AttachRolePolicy` 작업을 허용하는 정책으로 역할을 업데이트합니다.

SageMaker Studio 노트북을 사용하지 않는 경우, AWS CLI를 구성하는 데 사용한 역할에 IAM 역할을 생성하고 정책을 연결할 수 있는 적절한 권한이 있는지 확인하세요.

다음은 역할에 추가할 수 있는 인라인 정책의 예시입니다:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Statement1",
            "Effect": "Allow",
            "Action": [
                "iam:ListPolicies",
                "iam:CreatePolicy",
                "iam:CreateRole",
                "iam:AttachRolePolicy"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

In [ ]:
role_name = role.split('/')[-1]

print(f"현재 SageMaker 실행 역할: {role_name}")

# MLflow 권한 정책 생성 (AccessUI 권한 추가)
mlflow_policy_name = "sagemaker-mlflow-permissions"
mlflow_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*"
            ],
            "Resource": f"arn:aws:sagemaker:{region}:{account_id}:mlflow-tracking-server/*"
        }
    ]
}

# 정책 생성 또는 기존 정책 업데이트
try:
    # 정책 생성 시도
    mlflow_iam_policy = iam_client.create_policy(
        PolicyName=mlflow_policy_name, 
        PolicyDocument=json.dumps(mlflow_policy_document)
    )
    mlflow_policy_arn = mlflow_iam_policy["Policy"]["Arn"]
    print(f"새 정책 생성됨: {mlflow_policy_name}")
except iam_client.exceptions.EntityAlreadyExistsException:
    # 정책이 이미 존재하는 경우, 버전 업데이트
    mlflow_policy_arn = f"arn:aws:iam::{account_id}:policy/{mlflow_policy_name}"
    
    # 기존 정책 버전 확인
    policy_versions = iam_client.list_policy_versions(PolicyArn=mlflow_policy_arn)
    
    # 버전이 5개 이상이면 가장 오래된 버전(기본 버전 아닌) 삭제
    if len(policy_versions['Versions']) >= 5:
        for version in policy_versions['Versions']:
            if not version['IsDefaultVersion']:
                iam_client.delete_policy_version(
                    PolicyArn=mlflow_policy_arn,
                    VersionId=version['VersionId']
                )
                print(f"오래된 정책 버전 {version['VersionId']} 삭제됨")
                break
    
    # 새 버전 생성
    iam_client.create_policy_version(
        PolicyArn=mlflow_policy_arn,
        PolicyDocument=json.dumps(mlflow_policy_document),
        SetAsDefault=True
    )
    print(f"기존 정책 {mlflow_policy_name} 업데이트됨")

# 역할에 정책이 이미 연결되어 있는지 확인
attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
policy_already_attached = False

for policy in attached_policies.get('AttachedPolicies', []):
    if policy['PolicyArn'] == mlflow_policy_arn:
        policy_already_attached = True
        print(f"정책 {mlflow_policy_name}이(가) 이미 역할 {role_name}에 연결되어 있습니다.")
        break

# 정책이 연결되어 있지 않은 경우 연결
if not policy_already_attached:
    iam_client.attach_role_policy(
        RoleName=role_name, 
        PolicyArn=mlflow_policy_arn
    )
    print(f"정책 {mlflow_policy_name}을(를) 역할 {role_name}에 연결했습니다.")
    print("이제 MLflow UI 및 기타 작업을 수행할 수 있는 권한이 있습니다.")
else:
    print("MLflow 작업을 수행할 수 있는 권한이 이미 있습니다.")

print("\n권한 변경이 적용되는 데 몇 분 정도 소요될 수 있습니다.")

In [ ]:
mlflow_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": ["sagemaker.amazonaws.com"]},
            "Action": "sts:AssumeRole",
        }
    ],
}

# Create role for MLflow (if it doesn't exist)
try:
    mlflow_role = iam_client.create_role(
        RoleName=mlflow_role_name, AssumeRolePolicyDocument=json.dumps(mlflow_trust_policy)
    )
    mlflow_role_arn = mlflow_role["Role"]["Arn"]
    print(f"Created new role: {mlflow_role_name}")
except iam_client.exceptions.EntityAlreadyExistsException:
    # Role already exists, get its ARN
    mlflow_role = iam_client.get_role(RoleName=mlflow_role_name)
    mlflow_role_arn = mlflow_role["Role"]["Arn"]
    print(f"Using existing role: {mlflow_role_name}")

# Create policy for S3 and SageMaker Model Registry (if it doesn't exist)
policy_name = "mlflow-s3-sm-model-registry"
sm_s3_model_registry_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:Get*",
                "s3:Put*",
                "s3:List*",
                "sagemaker:AddTags",
                "sagemaker:CreateModelPackageGroup",
                "sagemaker:CreateModelPackage",
                "sagemaker:UpdateModelPackage",
                "sagemaker:DescribeModelPackageGroup",
            ],
            "Resource": "*",
        }
    ],
}

try:
    # Try to create the policy
    mlflow_s3_sm_model_registry_iam_policy = iam_client.create_policy(
        PolicyName=policy_name, PolicyDocument=json.dumps(sm_s3_model_registry_policy)
    )
    mlflow_s3_sm_model_registry_iam_policy_arn = mlflow_s3_sm_model_registry_iam_policy["Policy"]["Arn"]
    print(f"Created new policy: {policy_name}")
except iam_client.exceptions.EntityAlreadyExistsException:
    # Policy already exists, get its ARN
    account_id = boto3.client('sts').get_caller_identity().get('Account')
    mlflow_s3_sm_model_registry_iam_policy_arn = f"arn:aws:iam::{account_id}:policy/{policy_name}"
    print(f"Using existing policy: {policy_name}")

# Check if the policy is already attached to the role
attached_policies = iam_client.list_attached_role_policies(RoleName=mlflow_role_name)
policy_already_attached = False

for policy in attached_policies.get('AttachedPolicies', []):
    if policy['PolicyArn'] == mlflow_s3_sm_model_registry_iam_policy_arn:
        policy_already_attached = True
        print(f"Policy {policy_name} is already attached to role {mlflow_role_name}")
        break

# Attach the policy to the MLflow role if not already attached
if not policy_already_attached:
    iam_client.attach_role_policy(
        RoleName=mlflow_role_name, PolicyArn=mlflow_s3_sm_model_registry_iam_policy_arn
    )
    print(f"Attached policy {policy_name} to role {mlflow_role_name}")

참고: SageMaker 실행 역할은 Mlflow REST API를 호출하기 위해 다음 권한이 있어야 합니다:

```json
{
    "Version": "2012-10-17",    
    "Statement": [        
        {            
            "Effect": "Allow",            
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],            
            "Resource": "*"        
        }        
    ]
}
```

## MLflow Tracking Server 생성

In [ ]:
def check_mlflow_server_exists(server_name):
    """MLflow 서버가 존재하는지 확인하는 함수"""
    try:
        response = sm_client.describe_mlflow_tracking_server(
            TrackingServerName=server_name
        )
        # 서버가 존재하면 상태 반환
        return True, response['TrackingServerStatus']
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFound':
            # 서버가 존재하지 않음
            return False, None
        else:
            # 다른 오류 발생
            raise e

# MLflow 서버가 존재하는지 확인
server_exists, server_status = check_mlflow_server_exists(tracking_server_name)

if server_exists:
    print(f"MLflow 서버 '{tracking_server_name}'이(가) 이미 존재합니다. 상태: {server_status}")
else:
    print(f"MLflow 서버 '{tracking_server_name}'을(를) 생성합니다...")
    try:
        response = sm_client.create_mlflow_tracking_server(
            TrackingServerName=tracking_server_name,
            ArtifactStoreUri=f"s3://{bucket_name}/{tracking_server_name}",
            TrackingServerSize="Small",
            MlflowVersion="2.13.2",
            RoleArn=mlflow_role_arn,
            AutomaticModelRegistration=False,
        )
        
        print(f"MLflow 서버 생성 요청이 제출되었습니다. ARN: {response['TrackingServerArn']}")
        
        # 선택적: 서버가 활성화될 때까지 대기
        print("서버가 활성화될 때까지 대기 중...")
        while True:
            _, current_status = check_mlflow_server_exists(tracking_server_name)
            print(f"현재 상태: {current_status}")
            
            if current_status == 'InService':
                print("MLflow 서버가 성공적으로 생성되었습니다!")
                break
            elif current_status in ['Failed', 'Deleting']:
                print(f"MLflow 서버 생성 실패: {current_status}")
                break
            
            time.sleep(120)  # 30초마다 상태 확인
            
    except ClientError as e:
        print(f"MLflow 서버 생성 중 오류 발생: {e}")

In [ ]:
tracking_server_arn = (
    f"arn:aws:sagemaker:{region}:{account_id}:mlflow-tracking-server/{tracking_server_name}"
)

In [ ]:
sm_client.describe_mlflow_tracking_server(TrackingServerName=tracking_server_name)

MLflow SDK와 MLflow AWS 플러그인을 설치하세요.

In [ ]:
!pip install --quiet mlflow==2.13.2 sagemaker-mlflow==0.1.0

## MLflow tracking 테스트

tracking server 에 연결

In [ ]:
import mlflow

mlflow.set_tracking_uri(tracking_server_arn)

Log a metric

In [ ]:
with mlflow.start_run():
    mlflow.log_metric("foo", 1)

MLflow UI에서 결과를 확인하세요. SageMaker Studio 내에서 MLflow UI를 실행하거나, 다음과 같이 미리 서명된 URL을 생성할 수 있습니다:

In [ ]:
sm_client.create_presigned_mlflow_tracking_server_url(TrackingServerName=tracking_server_name)

In [ ]:
%store tracking_server_name